# Term Deposits - CashFlows

This notebook demonstrates how to access and use the functionalities of **Term Deposits CashFlows** within **LSEG Financial Analytics SDK**. 

The notebook outlines a simple method of calculation CashFlows for a set of term deposits instruments.

**You will be able to:**
- Define the term deposits instruments by tailoring its features
- Estimate future CashFlows


## Imports

Import the following necessary modules:

- `TermDepositDefinition` - Define the underlying instrument
- `TermDepositPricingParameters` - Define the underlying instrument
- `price` - Configure volatility model and settings  

In [1]:
from lseg_analytics.instruments import term_deposit as td

import datetime as dt
import json
import pandas as pd
from IPython.display import display

## Data Preparation


To define a Term Deposit instrument you need to follow a structured 3-step process:
1. **Term Deposit Definition** - Specify the term deposit definition attributes
2. **Term Deposit Instrument Definition** - Create the instrument object
3. **Pricing Preferences** - Configure pricing parameters, optional

We take here an example of 4% GBP Term deposit priced in historical time (valuation date is 2025-07-21)

In [2]:
# Create term deposit defintion object
term_deposit_definition = td.TermDepositDefinition(
    end_date = dt.datetime.strptime("2026-05-10", "%Y-%m-%d"),
    notional_ccy = "GBP", 
    notional_amount = 1000000,
    start_tenor = "2M",
    fixed_rate_percent = 4.0
)

# Create term deposit instrument defintion object
term_deposit_instrument = td.TermDepositDefinitionInstrument(
    definition = term_deposit_definition
)

# Create term deposit pricing parameters object - optional
term_deposit_parameters = td.TermDepositPricingParameters(
    valuation_date  = dt.datetime.strptime("2025-07-21", "%Y-%m-%d"),
    report_ccy = "USD"
)

## Request Execution

In [3]:
#  Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

term_deposit_response = td.price(
    definitions  =[term_deposit_instrument], 
    pricing_preferences = term_deposit_parameters
)

## Results Display

The results showed here are focused on term deposit cash-flow construction. We then give all necessary outputs: 
- description 
- pricing analysis
- cash-flows

In [4]:
# Access the description object
description = term_deposit_response.data.analytics[0].description
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentDescription": "Deposit 1000000 'GBP' 4% '2026-05-10'",
    "notionalCcy": "GBP",
    "notionalAmount": 1000000.0,
    "reportCcy": "USD",
    "startDate": "2025-09-23T00:00:00Z",
    "endDate": "2026-05-10T00:00:00Z",
    "interestCalculationMethod": "Dcb_Actual_365"
}


In [5]:
# Access the pricing analysis object
pricing_analysis = term_deposit_response.data.analytics[0].pricing_analysis
print(json.dumps(pricing_analysis.as_dict(), indent=4))

{
    "fixedRatePercent": 4.0,
    "valuationDate": "2025-07-21T00:00:00Z",
    "marketDataDate": "2025-07-21T00:00:00Z"
}


In [6]:
# Access to payments/cash-flows
payments = term_deposit_response.data.analytics[0].cashflows.cash_flows[0]["payments"]
df_payments = pd.DataFrame(payments)
display(df_payments)

,date,amount,remainingNotional,currency,event,type,occurence
0,2025-09-23,-1000000.000000,1000000.0,GBP,Principal,Fixed,Future
1,2026-05-11,25095.890411,0.0,GBP,Interest,Fixed,Future
2,2026-05-11,1000000.000000,0.0,GBP,Principal,Fixed,Future


Three cash-flows are returned:
- The first cash-flow is the notional amount lent to the other party at start date
- The second cash-flow is the interest amount received at end date
- The third cash-flow is the notional amount paid back at end date 

The first and third cash-flows are notional amount specified in inputs. The second cash-flow is the interest cash-flow. It is derived according to the following formula:
$$CashSettlement :=Notional\cdot FixedRatePercent\cdot DayCountFraction\left(StartDate, EndDate\right)$$
In this use-case, day count convention is Act/365, meaning that it is the number of days (here 79) divided by 365. We apply below this formula to explain its construction

In [7]:
ir_cash_flow_amount = payments[1]["amount"]
nb_days = nb_days = (description.end_date - description.start_date).days
computed_ir_cash_flow_amount = description.notional_amount * (pricing_analysis.fixed_rate_percent / 100) * (nb_days / 365)
print("Computed interest cash-flow:", computed_ir_cash_flow_amount)
print("Interest cash-flow returned by pricer:", ir_cash_flow_amount)
print("Difference:", computed_ir_cash_flow_amount - ir_cash_flow_amount)

Computed interest cash-flow: 25095.890410958902
Interest cash-flow returned by pricer: 25095.890410958902
Difference: 0.0
